In [10]:
import os
import csv
import shutil
import matplotlib.pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras import optimizers
from keras.applications import VGG16
from PIL import Image



In [11]:
#create the convolutional base 
convolutionalBase = VGG16( weights = 'imagenet',
                           include_top = False,
                           input_shape = ( 150, 150, 3 ) )
convolutionalBase.summary()

#create the dense layer to stick on top of the base
sequentialTop = Sequential()
sequentialTop.add( Dense( 256, activation = 'relu', input_dim = 8192 ) )
sequentialTop.add( Dropout( 0.5 ) )
sequentialTope.add( Dense( 4, activation = 'sigmoid' ) )

#compile the sequential top
sequentialTop.compile( optimizer = 'rmsprop',
                       loss = 'categorical_crossentropy',
                       metrics = [ 'acc' ] )





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

TypeError: __init__() missing 1 required positional argument: 'units'